## Vectorize İşlemi Nedir?

Metin verilerini makine öğrenmesi modellerine besleyebilmek için sayısal forma dönüştürme işlemine **vektörizasyon** denir. Ham metin (string) formatındaki veriler, modeller tarafından anlaşılamaz; bu nedenle her kelime veya n-gram özellik olarak sayısal bir değere sahip bir vektöre dönüştürülür.

---

## TF-IDF Nedir?

**TF-IDF (Term Frequency-Inverse Document Frequency)**, kelimelerin belgeler içindeki önemini ölçen yaygın bir vektörizasyon yöntemidir. İki temel bileşeni vardır:

- **TF (Term Frequency)**: Bir belgedeki bir kelimenin ne sıklıkla göründüğünü ölçer.
- **IDF (Inverse Document Frequency)**: Kelimenin tüm belgeler arasında ne kadar nadir olduğunu ölçer.

TF-IDF skoru, bir kelimenin belgede sık geçtiği ve aynı zamanda diğer belgelerde nadiren kullanıldığı durumları öne çıkarır.

---

## Formüller

- **Term Frequency (TF)**

  \[
    TF(t, d) = \frac{f_{t,d}}{\sum_{k} f_{k,d}}
  \]

  - *f<sub>t,d</sub>*: Belge *d* içindeki *t* teriminin frekansı.
  - Pay kısım tüm terimlerin toplam frekansı.

- **Inverse Document Frequency (IDF)**

  \[
    IDF(t, D) = \log\left(\frac{N}{1 + |\{d \in D: t \in d\}|}\right)
  \]

  - *N*: Toplam belge sayısı.
  - *|\{d \in D: t \in d\}|*: *t* terimini içeren belge sayısı.

- **TF-IDF**

  \[
    TF\text{-}IDF(t, d, D) = TF(t,d) \times IDF(t,D)
  \]

---

## Örnek Kullanım

```python
from sklearn.feature_extraction.text import TfidfVectorizer

# Örnek metin koleksiyonu (corpus)
corpus = [
    'Bu bir örnek cümle',
    'TF-IDF vektörleştirme işlemi',
    'Makine öğrenmesi için metin ön işleme'
]

# Vectorizer nesnesi oluşturma
vectorizer = TfidfVectorizer()

# fit ve transform işlemi
X = vectorizer.fit_transform(corpus)

# Özellik isimleri (vokabüloryum)
feature_names = vectorizer.get_feature_names_out()
print('Özellikler:', feature_names)

# TF-IDF matrisini dizi olarak gösterme
dense_matrix = X.toarray()
print('TF-IDF Matrisi:
', dense_matrix)
```


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Örnek metin koleksiyonu (corpus)
corpus = [
    'Bu bir örnek cümle',
    'TF-IDF vektörleştirme işlemi',
    'Makine öğrenmesi için metin ön işleme'
]

# Vectorizer nesnesi oluşturma
vectorizer = TfidfVectorizer()

# fit ve transform işlemi
X = vectorizer.fit_transform(corpus)

# Özellik isimleri (vokabüloryum)
feature_names = vectorizer.get_feature_names_out()
print('Özellikler:', feature_names)

# TF-IDF matrisini dizi olarak gösterme
dense_matrix = X.toarray()
print('TF-IDF Matrisi:\n', dense_matrix)

In [2]:
import pandas as pd
df = pd.read_csv('spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


In [3]:
df = df[["text", "label_num"]]
df.head()

,text,label_num
0,Subject: enron methanol ; meter # : 988291\nth...,0
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\nthis deal is to ...,0


In [4]:
import string
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()

def clean_text(text:str):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [word for word in words if not word.isdigit()]
    words = [ps.stem(word) for word in words if word not in stopwords.words("english")]
    return " ".join(words)

In [ ]:
df["clean_text"] = df["text"].apply(clean_text)
df.head()

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\nth...,0,subject enron methanol meter follow note gave ...
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0,subject hpl nom januari see attach file hplnol...
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0,subject neon retreat ho ho ho around wonder ti...
3,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window offic cheap main tren...
4,Subject: re : indian springs\nthis deal is to ...,0,subject indian spring deal book teco pvr reven...


In [8]:
import nltk
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

X = df["clean_text"]
y = df["label_num"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("model", MultinomialNB())
])

param_grid = {
    # 1-gram, 2-gram ve 3-gram deniyoruz
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    # En az 2, 5 veya 10 dokümanda geçen kelimeler
    'tfidf__min_df': [2, 5, 10],
    # En çok dokümanların %75, %85 veya %95’inde geçen kelimeleri at
    'tfidf__max_df': [0.75, 0.85, 0.95],
    # En fazla 2k, 5k, 10k veya bütün kelimeler
    'tfidf__max_features': [2000, 5000, 10000, None],
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

grid.fit(X_train, y_train)

print("Best parameters found: ", grid.best_params_)
print("Best cross-validation score: ", grid.best_score_)
print("Test set score: ", grid.score(X_test, y_test))


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'tfidf__max_df': 0.75, 'tfidf__max_features': None, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}
Best cross-validation score:  0.9666354146586521
Test set score:  0.9594202898550724
